<span type="title">流水线处理器</span> | <span type="update">2018-07-16</span> | <span type="version">1</span>

<span type="intro"><p type="card-text">本章主要介绍流水线处理器。</p></span>

# 基本原理

传统MIPS的执行指令主要分为五步，取指（存储器取指令，更新PC）、译码（寄存器堆读寄存器值）、执行（运算指令使用ALU，访存指令计算存储器地址）、访存（Load指令从储存器读数据，Store数据写入储存器），回写（将数据写入寄存器堆）。

![](w6p1.png)

如图的蓝色部分就是流水线寄存器所在位置，当添加流水线寄存器之后，在五个阶段进行的中间数据就得以保存，这样的话，当取指完成其功能后就进行下一条指令，**节省了时间，提高了时间连续性和吞吐率。**

如果一条指令需要1000ps，每个部分需要200ps，那么对于单周期处理器而言，时钟周期为1000ps，对于流水线处理器而言，时钟周期为200ps。

**那么，可以无限的划分流程，而无限的提高频率吗？**实际情况是，因为流水寄存器的消耗，每个部分会出现延迟，当划分段数越多，延迟越多。但是因为划分段数越多，指令频率越高，账面越好看，但是实际执行时间会随着每部分流水线组件交接的延时而拖累整体的执行速度。



流水线的第二个问题是，如果每个阶段的运行时间不相等，那么就要以最慢的那个阶段为时钟频率。因此，尽可能的划分